In [1]:
import netsquid as ns
import pydynaa
ns.set_random_state(seed=42)

In [2]:
class PingEntity(pydynaa.Entity):
    ping_evtype = pydynaa.EventType("PING_EVENT", "A ping event.")
    delay = 10.

    def start(self, qubit):
        # Start the game by scheduling the first ping event after delay
        self.qubit = qubit
        self._schedule_after(self.delay, PingEntity.ping_evtype)

    def wait_for_pong(self, pong_entity):
        # Setup this entity to listen for pong events from a PongEntity
        pong_handler = pydynaa.EventHandler(self._handle_pong_event)
        self._wait(pong_handler, entity=pong_entity,
                   event_type=PongEntity.pong_evtype)

    def _handle_pong_event(self, event):
        # Callback function called by the pong handler when pong event is triggered
        m, prob = ns.qubits.measure(self.qubit, observable=ns.Z)
        labels_z = ("|0>", "|1>")
        print(f"{ns.sim_time():.1f}: Pong event! PingEntity measured "
              f"{labels_z[m]} with probability {prob:.2f}")
        self._schedule_after(PingEntity.delay, PingEntity.ping_evtype)

class PongEntity(pydynaa.Entity):
    pong_evtype = pydynaa.EventType("PONG_EVENT", "A pong event.")
    delay = 10.

    def wait_for_ping(self, ping_entity):
        # Setup this entity to listen for ping events from a PingEntity
        ping_handler = pydynaa.EventHandler(self._handle_ping_event)
        self._wait(ping_handler, entity=ping_entity,
                   event_type=PingEntity.ping_evtype)

    def _handle_ping_event(self, event):
        # Callback function called by the ping handler when ping event is triggered
        m, prob = ns.qubits.measure(event.source.qubit, observable=ns.X)
        labels_x = ("|+>", "|->")
        print(f"{ns.sim_time():.1f}: Ping event! PongEntity measured "
              f"{labels_x[m]} with probability {prob:.2f}")
        self._schedule_after(PongEntity.delay, PongEntity.pong_evtype)

In [3]:
ping = PingEntity()
pong = PongEntity()
ping.wait_for_pong(pong)
pong.wait_for_ping(ping)

In [5]:
qubit, = ns.qubits.create_qubits(1)
ping.start(qubit)

In [6]:
stats = ns.sim_run(end_time=91)

10.0: Ping event! PongEntity measured |+> with probability 0.50
20.0: Pong event! PingEntity measured |1> with probability 0.50
30.0: Ping event! PongEntity measured |-> with probability 0.50
40.0: Pong event! PingEntity measured |1> with probability 0.50
50.0: Ping event! PongEntity measured |+> with probability 0.50
60.0: Pong event! PingEntity measured |0> with probability 0.50
70.0: Ping event! PongEntity measured |+> with probability 0.50
80.0: Pong event! PingEntity measured |1> with probability 0.50
90.0: Ping event! PongEntity measured |-> with probability 0.50


In [7]:
print(stats)


Simulation summary

Elapsed wallclock time: 0:00:00.002523
Elapsed simulation time: 9.10e+01 [ns]
Triggered events: 9
Handled callbacks: 9
Total quantum operations: 9
Frequent quantum operations: MEASURE = 9
Max qstate size: 1 qubits
Mean qstate size: 1.00 qubits



### Event Expressions by example: quantum teleportation

In [ ]:
class Charlie(pydynaa.Entity):
    ready_evtype = pydynaa.EventType("QUBITS_READY", "Entangled qubits are ready.")
    _generate_evtype = pydynaa.EventType("GENERATE", "Generate entangled qubits.")
    period = 50.
    delay = 10.

    def __init__(self):
        # Initialise Charlie by entangling qubits after every generation event
        self.entangled_qubits = None
        self._generate_handler = pydynaa.EventHandler(self._entangle_qubits)
        self._wait(self._generate_handler, entity=self,
                   event_type=Charlie._generate_evtype)

    def _entangle_qubits(self, event):
        # Callback function that entangles qubits and schedules an
        # entanglement ready event
        q1, q2 = ns.qubits.create_qubits(2)
        ns.qubits.operate(q1, ns.H)
        ns.qubits.operate([q1, q2], ns.CNOT)
        self.entangled_qubits = [q1, q2]
        self._schedule_after(Charlie.delay, Charlie.ready_evtype)
        print(f"{ns.sim_time():.1f}: Charlie finished generating entanglement")
        self._schedule_after(Charlie.period, Charlie._generate_evtype)

    def start(self):
        # Begin generating entanglement
        print(f"{ns.sim_time():.1f}: Charlie start generating entanglement")
        self._schedule_now(Charlie._generate_evtype)